In [65]:
import numpy as np
import torch

X = torch.rand([4, 3]) # N * D = 4 * 3
W1 = torch.rand([3, 2]) # D * H = 3 * 2
b1 = torch.rand([4, 1]) # N * 1 = 4
W2 = torch.rand([2, 5]) # H * C = 2 * 5
b2 = torch.rand([4, 1]) # N * 1 = 4
y = torch.randperm(4) # N * 1 = 4
reg = 0.01

N, D = X.shape
D, H = W1.shape
C = W2.shape[1]
#a = np.random.rand(4,3)
#a = np.array(np.random.rand(3,2))

H0 = X.mm(W1) + b1 # N * H = 4 * 2
# ReLU of 1st hidden
hidden = torch.maximum(torch.zeros_like(H0), H0) # N * H = 4 * 2

scores = hidden.mm(W2) + b2 # N * C = 4 * 5

print(H0, '\n', hidden, '\n\n', 'scores =\n', scores)
print('\n', y)

tensor([[1.9289, 1.5684],
        [1.2898, 1.0813],
        [1.4758, 1.2528],
        [1.3474, 0.8687]]) 
 tensor([[1.9289, 1.5684],
        [1.2898, 1.0813],
        [1.4758, 1.2528],
        [1.3474, 0.8687]]) 

 scores =
 tensor([[2.1572, 2.2741, 2.7353, 0.5950, 1.5344],
        [1.6569, 1.7590, 2.0642, 0.6128, 1.2395],
        [2.1632, 2.2915, 2.6391, 0.9688, 1.6851],
        [1.7357, 1.6509, 1.9958, 0.6411, 1.3079]])

 tensor([1, 0, 2, 3])


In [76]:
# Forwardprop
num = torch.arange(N)
denominator = torch.sum(torch.exp(scores), axis = 1, keepdim = True) # N * 1 = 4 * 1
softmax = torch.exp(scores) / denominator # N * C = 4 * 5
correct_class = softmax[num, y]
#print(softmax, correct_class)
loss = torch.sum(-torch.log(correct_class)) / N
loss += reg * (torch.sum(W1 * W1) + torch.sum(W2 * W2))


# Backprop
grad = {}

# last layer
softmax[num, y] -= 1
#print(softmax)
grad['W2'] = hidden.t().mm(softmax) # H * C = 2 * 5
grad['b2'] = denominator * C - 1 # N * 1 = 4 * 1
grad['hidden'] = softmax.mm(W2.t()) # N * H = 4 * 2

# 2nd last layer: ReLU
# refer: https://stackoverflow.com/questions/58002836/pytorch-1-if-x-0-5-else-0-for-x-in-outputs-with-tensors
grad_ReLU = (hidden > 0).float()
#print(grad_ReLU)
grad['hidden'] *= grad_ReLU # N * H = 4 * 2
#print(grad['hidden'], '\n')

# input layer
grad['W1'] = X.t().mm(grad['hidden']) # D * H = (D * N) (N * H) = 3 * 2
grad['b1'] = torch.sum(grad['hidden'], axis = 1, keepdim = True) # N * 1
#print(grad['b1'], '\n', b1)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])
